In [ ]:
import math
import cv2
import numpy as np

In [ ]:
image = cv2.imread('/home/julia/TFM/CenterNet/models/2.jpg')
window_size = 512

In [ ]:
def crop_image_sliding_window(image, overlap_degree=0.3, window_size=512):
  height, width = image.shape[0:2]
  overlap = math.ceil(window_size * overlap_degree)
  step = window_size - overlap
  
  for y in range(0, height, step):
    for x in range(0, width, step):    
      y2 = y + window_size
      x2 = x + window_size
      zeros_image = np.zeros([window_size, window_size, 3], dtype=np.uint8)
      if y2 > height:
        y2 = height
      if x2 > width:
        x2 = width
      cropped_image = image[y:y2, x:x2]
      result_image = zeros_image.copy()
      result_image[:cropped_image.shape[0], :cropped_image.shape[1], :] = cropped_image
      yield (x, x2, y, y2, result_image)

In [ ]:
# crop numpy images with sliding window

image = cv2.imread('/home/julia/TFM/CenterNet/data/barcode/test/2.jpg')
for (x1, x2, y1, y2, cropped_image) in crop_image_sliding_window(image):
    cv2.imshow('cropped_image', cropped_image)
    cv2.waitKey(0)
    print('bbox: ', x1, x2, y1, y2)

# cv2.destroyAllWindows

In [ ]:
def crop_image_sliding_window(image, window_size=512):
  height, width = image.shape[0:2]
  y_windows = height/window_size # 3
  x_windows = width/window_size

  round_y_windows = math.floor(y_windows + 1) # 4
  round_x_windows = math.floor(x_windows + 1)
  step_size_y = (round_y_windows - y_windows ) * window_size # 512
  step_size_x = (round_x_windows - x_windows) * window_size

  step_size_y = math.ceil(step_size_y/math.floor(y_windows)) # 170
  step_size_x = math.ceil(step_size_x/math.floor(x_windows)) # 128
  
  y1 = 0
  x1 = 0
  for y in range(0, round_y_windows):
    prev_y = y1 + window_size
    for x in range(0, round_x_windows):
      prev_x = x1 + window_size
      if x == 0 and y == 0:
        y2 = y1 + window_size
        x2 = x1 + window_size
        cropped_image = image[y1:y2, x1:x2]
      elif x != 0 and y != 0:
        y1 = prev_y - step_size_y
        x1 = prev_x - step_size_x
        y2 = y1 + window_size
        x2 = x1 + window_size
        cropped_image = image[y1:y2, x1:x2]
      elif x == 0:
        y1 = prev_y - step_size_y
        x1 = 0
        y2 = y1 + window_size
        x2 = x1 + window_size       
        cropped_image = image[y1:y2, x1:x2]
      else:
        y1 = 0
        x1 = prev_x - step_size_x
        y2 = y1 + window_size
        x2 = x1 + window_size
        cropped_image = image[y1:y2, x1:x2]
      yield (x1, x2, y1, y2, cropped_image) 